# Simulated-based-inference of 2D Epileptor model

In [1]:
import os
import sys
import time
import errno
import time
import timeit


import numpy as np
import scipy as scp


import numba
from numba import jit

In [2]:
from scipy import stats as spstats
from scipy.stats import moment
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import mode

from scipy.optimize import fsolve
from scipy.optimize import root

In [3]:
import torch
import sbi 
import sbi.utils as utils
from sbi.inference.base import infer

#from sbi.inference import SNPE, SNLE, SNRE, prepare_for_sbi
#from sbi.utils.get_nn_models import posterior_nn

In [4]:
%pylab inline
#%matplotlib notebook

Populating the interactive namespace from numpy and matplotlib


In [5]:
plt.style.use('seaborn-talk');
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = 'Helvetica'
plt.rcParams['axes.edgecolor']='#333F4B'
plt.rcParams['axes.linewidth']=0.8
plt.rcParams['xtick.color']='#333F4B'
plt.rcParams['ytick.color']='#333F4B'

In [6]:
cwd = os.getcwd()
cwd

'/data/SBI/BVEP_SBI'

In [7]:
Res_dir='Res_Epileptor2D_sde_sbi_fitstatsummary_LargeTau'

### Simulator

In [8]:
from Epileptor_Simulator import Epileptor2Dmodel, Epileptor6Dmodel

In [10]:
def root_fuc(roots, eta):   
    xx = np.empty(1)
    zz = np.empty(1)
    F = np.empty(2)
    xx = roots[0]
    zz = roots[1]
    I1=3.1
    F[0] =1.0 - xx**3 - 2.0*xx**2 - zz + I1 
    F[1] =4*(xx-eta) - zz
    return np.array([F[0], F[1]])
######################################################    
def nullclines(X,Y, tau, eta):  
    I1=3.1
    dX =  1.0 - X**3 - 2.0*X**2 - Y + I1 
    dY =  4*(X-eta) - Y
    plt.contour(X, Y, dX, 0, colors='b', linewidths=(2.,))
    plt.contour(X, Y, dY, 0, colors='b', linewidths=(2.,))
    
    x = np.r_[-5:5:0.01]
    z = np.r_[0:10:0.01]
    xx, zz = np.meshgrid(x,z)
    dx = np.zeros([len(z),len(x)])
    dz = np.zeros([len(z),len(x)])

    for ii in range(len(z)):
        for jj in range(len(x)):
            dx[ii,jj] = 1 - x[jj]**3 - 2*x[jj]**2 - z[ii] + I1
            dz[ii,jj] = 1/tau*(4*(x[jj] - eta) - z[ii])
    plt.streamplot(x,z,dx,dz,density=2.0, color='deepskyblue', linewidth=1.5 , zorder=1) 

In [11]:
# X, Z = mgrid[-5.0:5.0:100j, 0.0:10.0:100j]

# rGuess=array([[-1.,3.]])

In [12]:
# eta_true=-1.5
# tau_true=10.
# true_roots = fsolve(root_fuc,rGuess, args=(eta_true,))

In [13]:
# plt.figure(figsize=(5, 5))
# nullclines(X,Z, tau_true, eta_true)
# #plt.plot(true_roots[0], true_roots[1] , marker="o", markersize=10, color='k')
# plt.scatter(true_roots[0], true_roots[1], s=28, facecolors='k', edgecolors='k', linewidth=2., zorder=3)
# plt.xlabel('x'); plt.ylabel('z'); 
# plt.title('phase-plane')
# plt.xlim((-3,2)), plt.ylim((1,6))
# plt.show()

In [14]:
def calculate_summary_statistics(x, params, dt, ts, features):
    """Calculate summary statistics

    Parameters
    ----------
    x : output of the simulator

    Returns
    -------
    np.array, summary statistics
    """

    params.astype(float)

    n_summary = 10

    sum_stats_vec = np.concatenate((np.array([np.mean(x)]),
                                    np.array([np.std(x)]),
                                    np.array([skew(x)]),
                                    np.array([kurtosis(x)]),
                                    ))

    for item in features:

          if item is 'seizures_onset':

                    # initialise array of seizure counts
                    nt=ts.shape[0]
                    v=np.zeros(nt)
                    v= np.array(x)

                    v_th=0
                    ind = np.where(v < v_th)
                    v[ind] = v_th

                    ind = np.where(np.diff(v) < 0)
                    v[ind] = v_th

                    seizure_times = np.array(ts)[ind]
                    seizure_times_stim = seizure_times

                    if seizure_times_stim.shape[0] > 0:
                            seizure_times_stim = seizure_times_stim[np.append(1, np.diff(seizure_times_stim)) > 0.75]

        
                    sum_stats_vec = np.concatenate((sum_stats_vec, 
                                                    np.array([seizure_times_stim.shape[0]]),
                                                   ))

            
          if item is 'fixed_point':
                
                    rGuess=array([[-1.,3.]])
                    true_roots = fsolve(root_fuc,rGuess, args=(params[0],))

                    sum_stats_vec = np.concatenate((sum_stats_vec,
                                                    true_roots,
                                                       ))

    sum_stats_vec = sum_stats_vec[0:n_summary]     


    return sum_stats_vec


In [ ]:
def Epileptor2Dmodel_simulator_wrapper(params):

    params = np.asarray(params)
    
    # time step
    T = 100.0
    dt=0.1
    ts = np.arange(0, T + dt, dt)

    # fixed parameters
    I1=3.1
    constants = np.array([I1])
    
    sigma=1e-1
    nt=ts.shape[0]
    
    states = Epileptor2Dmodel(params, constants, sigma, dt, ts)

    summstats = torch.as_tensor(calculate_summary_statistics(states[0:nt], params, dt, ts, features=['seizures_onset']))

        
    return summstats

### sanity check simulations

### 2. Prior over model parameters

In [ ]:
prior_min = [-5.0, 0.1, -5.0, 0.0]
prior_max = [0.0, 50.0, 0.0, 5.0]
prior = utils.torchutils.BoxUniform(low=torch.as_tensor(prior_min), high=torch.as_tensor(prior_max))

### 3. Inference

In [ ]:
start_time = time.time()

posterior = infer(Epileptor2Dmodel_simulator_wrapper, prior, method='SNPE', num_simulations=10000, num_workers=1)

print ("-"*60)
print("--- %s seconds ---" % (time.time() - start_time))

### EZ

In [ ]:
# true parameters and respective labels
labels_params = [r'$\eta$', r'$\tau$', r'$x_{init}$', r'$z_{init}$']

In [ ]:
true_params = np.array([-1.6, 200., -2.5, 3.5])

In [ ]:
y_obs=Epileptor2Dmodel(true_params, constants, sigma, dt, ts)#+sigma*randn(2*ts.shape[0])

In [ ]:
x_obs, z_obs=y_obs[0:nt], y_obs[nt:2*nt]

In [ ]:
x_obs_summary_statistics=calculate_summary_statistics(x_obs, true_params, dt, ts, features=['seizures_onset', 'fixed_point'])

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(x_obs)
plt.title('observation')
plt.ylabel('fast-variable (x)', fontsize=18); 
plt.xlabel('Time', fontsize=18); 
plt.xticks(fontsize=16)
plt.xticks(fontsize=16)
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"obs_ez.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"obs_ez.eps"), doi=800)
plt.show()

In [ ]:
x_obs_summary_statistics

### Posterior EZ

In [ ]:
num_samples=10000

In [ ]:
posterior_samples = posterior.sample((num_samples,), x_obs_summary_statistics).numpy()

In [ ]:
np.save(os.path.join(cwd+'/'+str(Res_dir), 'Epileptor2D_sde_sbi_fitstatsummary_ez_posterior_samples.npy') , posterior_samples)

In [ ]:
num_params=posterior_samples.shape[1]

In [ ]:
plt.figure(figsize=(22,22))

for i in range(num_params):
    plt.subplot(2*num_params,2,2*i+1)
    plt.plot(posterior_samples[:,i], color='blue')
    plt.hlines(y=true_params[i], xmin=0., xmax=num_samples, linewidth=2.5, color = 'red', linestyle='--', zorder=5)
    plt.xlabel('number of samples',  fontsize=22)
    plt.ylabel(labels_params[i],  fontsize=24)
    plt.subplot(2*num_params,2,2*i+2)
    plt.hist(posterior_samples[:,i], bins=100, color='b')
    plt.vlines(x=true_params[i], ymin=0., ymax=200, linewidth=2.5, color = 'r', linestyle='--', zorder=5)
    plt.xlabel(r'$parameters$',  fontsize=22)
    plt.ylabel('Posterior ('+str(labels_params[i])+')',  fontsize=18)
#plt.tight_layout();   
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_obs_ez.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_obs_ez.eps"), doi=800)    
plt.show()

In [ ]:
fig, axes = utils.pairplot(posterior_samples,
                           fig_size=(8,8),
                           labels=labels_params,
                           points=true_params,
                           points_offdiag={'markersize': 20},
                           points_colors='r');
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_obs_ez.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_obs_ez.eps"), doi=800) 
plt.show()

In [ ]:
fig, axes = utils.pairplot(posterior_samples,
                           fig_size=(8,8),
                           labels=labels_params,
                           upper=['kde'],
                           diag=['kde'],
                           points=true_params,
                           points_offdiag={'markersize': 20},
                           points_colors='r');
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_kde_obs_ez.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_kde_obs_ez.eps"), doi=800) 
plt.show()

In [ ]:
y_fit=Epileptor2Dmodel(np.percentile(posterior_samples, [50], axis=0).reshape(-1), constants, sigma, dt, ts)

In [ ]:
x_fit, z_fit=y_fit[0:nt], y_fit[nt:2*nt]

In [ ]:
np.save(os.path.join(cwd+'/'+str(Res_dir), 'Epileptor2D_sde_sbi_fitstatsummary_ez_y_obs.npy') , y_obs)
np.save(os.path.join(cwd+'/'+str(Res_dir), 'Epileptor2D_sde_sbi_fitstatsummary_ez_y_fit.npy') , y_fit)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(x_obs, '--o', color='b', markersize=6,  label='obs')
plt.plot(x_fit, '-', color='r', lw=2, markersize=4,  label='fit')
plt.ylabel('fast-variable (x)', fontsize=24); 
plt.xlabel('Time', fontsize=24); 
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='upper right')
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_obs_ez.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_obs_ez.eps"), doi=800) 
plt.show()

In [ ]:
eta_true=true_params[0]
true_roots = fsolve(root_fuc,rGuess, args=(eta_true,))

In [ ]:
eta_est=np.mean(posterior_samples[:,0])
est_roots = fsolve(root_fuc,rGuess, args=(eta_est,))

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(x_obs, z_obs, '-o', color='darkred', alpha=1, linewidth=2., zorder=3, label='observation')
plt.plot(x_fit, z_fit, color='pink', alpha=1, linewidth=1.5, zorder=3,  label='predicted')
nullclines(X,Z,  tau_true, eta_true)
#plt.plot(true_roots_ez[0], true_roots_ez[1] , marker="o", markersize=14, color='cyan', label='true fixed point')
#plt.plot(est_roots_ez[0], est_roots_ez[1] , marker="o", markersize=14, color='y', label='est fixed point')
plt.scatter(true_roots[0],true_roots[1] , s=200, facecolors='none', edgecolors='k', linewidth=1., zorder=4, label='true fixed point') 
plt.scatter(est_roots[0], est_roots[1] , s=200, facecolors='none', linewidth=1., edgecolors='grey', zorder=5, label='est fixed point') 
plt.xlabel('fast-variable (x)', fontsize=28); 
plt.ylabel('slow-variable (z)', fontsize=28); 
plt.legend(loc='upper right')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.axis([-3, 1.5, 1.5, 5.5])
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_phaseplane_obs_ez.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_phaseplane_obs_ez.eps"), doi=800) 
plt.show()

### PZ

In [ ]:
true_params = np.array([-2.1, 10., -2.5, 3.5])

In [ ]:
y_obs=Epileptor2Dmodel(true_params, constants, sigma, dt, ts)#+sigma*randn(2*ts.shape[0])

In [ ]:
x_obs, z_obs=y_obs[0:nt], y_obs[nt:2*nt]

In [ ]:
x_obs_summary_statistics=calculate_summary_statistics(x_obs, true_params, dt, ts, features=['seizures_onset', 'fixed_point'])

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(x_obs)
plt.title('observation')
plt.ylabel('fast-variable (x)', fontsize=18); 
plt.xlabel('Time', fontsize=18); 
plt.xticks(fontsize=16)
plt.xticks(fontsize=16)
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"obs_pz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"obs_pz.eps"), doi=800)
plt.show()

In [ ]:
x_obs_summary_statistics

### Posterior PZ

In [ ]:
num_samples=10000

In [ ]:
posterior_samples = posterior.sample((num_samples,), x_obs_summary_statistics).numpy()

In [ ]:
np.save(os.path.join(cwd+'/'+str(Res_dir), 'Epileptor2D_sde_sbi_fitstatsummary_pz_posterior_samples.npy') , posterior_samples)

In [ ]:
num_params=posterior_samples.shape[1]

In [ ]:
plt.figure(figsize=(22,22))

for i in range(num_params):
    plt.subplot(2*num_params,2,2*i+1)
    plt.plot(posterior_samples[:,i], color='blue')
    plt.hlines(y=true_params[i], xmin=0., xmax=num_samples, linewidth=2.5, color = 'red', linestyle='--', zorder=5)
    plt.xlabel('number of samples',  fontsize=22)
    plt.ylabel(labels_params[i],  fontsize=24)
    plt.subplot(2*num_params,2,2*i+2)
    plt.hist(posterior_samples[:,i], bins=100, color='b')
    plt.vlines(x=true_params[i], ymin=0., ymax=200, linewidth=2.5, color = 'r', linestyle='--', zorder=5)
    plt.xlabel(r'$parameters$',  fontsize=22)
    plt.ylabel('Posterior ('+str(labels_params[i])+')',  fontsize=18)
#plt.tight_layout();   
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_obs_pz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_obs_pz.eps"), doi=800)    
plt.show()

In [ ]:
fig, axes = utils.pairplot(posterior_samples,
                           fig_size=(8,8),
                           labels=labels_params,
                           points=true_params,
                           points_offdiag={'markersize': 20},
                           points_colors='r');
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_obs_pz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_obs_pz.eps"), doi=800) 
plt.show()

In [ ]:
fig, axes = utils.pairplot(posterior_samples,
                           fig_size=(8,8),
                           labels=labels_params,
                           upper=['kde'],
                           diag=['kde'],
                           points=true_params,
                           points_offdiag={'markersize': 20},
                           points_colors='r');
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_kde_obs_pz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_kde_obs_pz.eps"), doi=800) 
plt.show()

In [ ]:
y_fit=Epileptor2Dmodel(np.percentile(posterior_samples, [50], axis=0).reshape(-1), constants, sigma, dt, ts)

In [ ]:
x_fit, z_fit=y_fit[0:nt], y_fit[nt:2*nt]

In [ ]:
np.save(os.path.join(cwd+'/'+str(Res_dir), 'Epileptor2D_sde_sbi_fitstatsummary_pz_y_obs.npy') , y_obs)
np.save(os.path.join(cwd+'/'+str(Res_dir), 'Epileptor2D_sde_sbi_fitstatsummary_pz_y_fit.npy') , y_fit)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(x_obs, '--o', color='b', markersize=6,  label='obs')
plt.plot(x_fit, '-', color='r', lw=2, markersize=4,  label='fit')
plt.ylabel('fast-variable (x)', fontsize=24); 
plt.xlabel('Time', fontsize=24); 
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='upper right')
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_obs_pz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_obs_pz.eps"), doi=800) 
plt.show()

In [ ]:
eta_true=true_params[0]
true_roots = fsolve(root_fuc,rGuess, args=(eta_true,))

In [ ]:
eta_est=np.mean(posterior_samples[:,0])
est_roots = fsolve(root_fuc,rGuess, args=(eta_est,))

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(x_obs, z_obs, '-o', color='darkgoldenrod', alpha=1, linewidth=2., zorder=3, label='observation')
plt.plot(x_fit, z_fit, color='gold', alpha=1, linewidth=1.5, zorder=3,  label='predicted')
nullclines(X,Z,  tau_true, eta_true)
plt.scatter(true_roots[0],true_roots[1] , s=200, facecolors='none', edgecolors='k', linewidth=1., zorder=4, label='true fixed point') 
plt.scatter(est_roots[0], est_roots[1] , s=200, facecolors='none', linewidth=1., edgecolors='grey', zorder=5, label='est fixed point') 
plt.xlabel('fast-variable (x)', fontsize=28); 
plt.ylabel('slow-variable (z)', fontsize=28); 
plt.legend(loc='upper right')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.axis([-3, 1.5, 1.5, 5.5])
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_phaseplane_obs_pz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_phaseplane_obs_p1.eps"), doi=800) 
plt.show()

### HZ

In [ ]:
true_params = np.array([-3., 10., -2.5, 3.5])

In [ ]:
y_obs=Epileptor2Dmodel(true_params, constants, sigma, dt, ts)#+sigma*randn(2*ts.shape[0])

In [ ]:
x_obs, z_obs=y_obs[0:nt], y_obs[nt:2*nt]

In [ ]:
x_obs_summary_statistics=calculate_summary_statistics(x_obs, true_params, dt, ts, features=['seizures_onset', 'fixed_point'])

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(x_obs)
plt.title('observation')
plt.ylabel('fast-variable (x)', fontsize=18); 
plt.xlabel('Time', fontsize=18); 
plt.xticks(fontsize=16)
plt.xticks(fontsize=16)
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"obs_hz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"obs_hz.eps"), doi=800)
plt.show()

In [ ]:
x_obs_summary_statistics

### Posterior HZ

In [ ]:
num_samples=10000

In [ ]:
posterior_samples = posterior.sample((num_samples,), x_obs_summary_statistics).numpy()

In [ ]:
np.save(os.path.join(cwd+'/'+str(Res_dir), 'Epileptor2D_sde_sbi_fitstatsummary_hz_posterior_samples.npy') , posterior_samples)

In [ ]:
num_params=posterior_samples.shape[1]

In [ ]:
plt.figure(figsize=(22,22))

for i in range(num_params):
    plt.subplot(2*num_params,2,2*i+1)
    plt.plot(posterior_samples[:,i], color='blue')
    plt.hlines(y=true_params[i], xmin=0., xmax=num_samples, linewidth=2.5, color = 'red', linestyle='--', zorder=5)
    plt.xlabel('number of samples',  fontsize=22)
    plt.ylabel(labels_params[i],  fontsize=24)
    plt.subplot(2*num_params,2,2*i+2)
    plt.hist(posterior_samples[:,i], bins=100, color='b')
    plt.vlines(x=true_params[i], ymin=0., ymax=200, linewidth=2.5, color = 'r', linestyle='--', zorder=5)
    plt.xlabel(r'$parameters$',  fontsize=22)
    plt.ylabel('Posterior ('+str(labels_params[i])+')',  fontsize=18)
#plt.tight_layout();   
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_obs_hz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_obs_hz.eps"), doi=800)    
plt.show()

In [ ]:
fig, axes = utils.pairplot(posterior_samples,
                           fig_size=(8,8),
                           labels=labels_params,
                           points=true_params,
                           points_offdiag={'markersize': 20},
                           points_colors='r');
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_obs_hz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_obs_hz.eps"), doi=800) 
plt.show()

In [ ]:
fig, axes = utils.pairplot(posterior_samples,
                           fig_size=(8,8),
                           labels=labels_params,
                           upper=['kde'],
                           diag=['kde'],
                           points=true_params,
                           points_offdiag={'markersize': 20},
                           points_colors='r');
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_kde_obs_hz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Posterior_pairplot_kde_obs_hz.eps"), doi=800) 
plt.show()

In [ ]:
y_fit=Epileptor2Dmodel(np.percentile(posterior_samples, [50], axis=0).reshape(-1), constants, sigma, dt, ts)

In [ ]:
x_fit, z_fit=y_fit[0:nt], y_fit[nt:2*nt]

In [ ]:
np.save(os.path.join(cwd+'/'+str(Res_dir), 'Epileptor2D_sde_sbi_fitstatsummary_hz_y_obs.npy') , y_obs)
np.save(os.path.join(cwd+'/'+str(Res_dir), 'Epileptor2D_sde_sbi_fitstatsummary_hz_y_fit.npy') , y_fit)

In [ ]:
plt.figure(figsize=(10,4))
plt.plot(x_obs, '--o', color='b', markersize=6,  label='obs')
plt.plot(x_fit, '-', color='r', lw=2, markersize=4,  label='fit')
plt.ylabel('fast-variable (x)', fontsize=24); 
plt.xlabel('Time', fontsize=24); 
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.legend(loc='upper right')
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_obs_hz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_obs_hz.eps"), doi=800) 
plt.show()

In [ ]:
eta_true=true_params[0]
true_roots = fsolve(root_fuc,rGuess, args=(eta_true,))

In [ ]:
eta_est=np.mean(posterior_samples[:,0])
est_roots = fsolve(root_fuc,rGuess, args=(eta_est,))

In [ ]:
plt.figure(figsize=(8,8))
plt.plot(x_obs, z_obs, '-o', color='darkgreen', alpha=1, linewidth=2., zorder=3, label='observation')
plt.plot(x_fit, z_fit, color='springgreen', alpha=1., linewidth=1.5, zorder=3,  label='predicted')
nullclines(X,Z,  tau_true, eta_true)
plt.scatter(true_roots[0],true_roots[1] , s=200, facecolors='k', edgecolors='k', linewidth=1., zorder=4, label='true fixed point') 
plt.scatter(est_roots[0], est_roots[1] , s=200, facecolors='grey', linewidth=1., edgecolors='grey', zorder=5, label='est fixed point') 
plt.xlabel('fast-variable (x)', fontsize=28); 
plt.ylabel('slow-variable (z)', fontsize=28); 
plt.legend(loc='upper right')
plt.xticks(fontsize=18)
plt.yticks(fontsize=18)
plt.axis([-3, 1.5, 1.5, 6.])
plt.tight_layout();
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_phaseplane_obs_hz.png"), doi=800)
plt.savefig(os.path.join(cwd+'/'+str(Res_dir),"Fitted_phaseplane_obs_hz.eps"), doi=800) 
plt.show()